In [2]:
import numpy as np
import math
from scipy.stats import norm, gamma, poisson

In [3]:
SeattleA = np.array([[15,12,4],[11,17,4],[0,21,4],[0,0,5]])
SeattleA

array([[15, 12,  4],
       [11, 17,  4],
       [ 0, 21,  4],
       [ 0,  0,  5]])

In [4]:
#
# Process data
#

def process(DATA):
    hsA = np.sum(DATA,axis=0)
    isA = np.sum(DATA,axis=1)
    colA = np.array(range(1,len(hsA)+1))
    rowA = np.array(range(0,len(hsA)+1))
    
    xA = np.sum(isA*rowA) # Final size
    N = np.sum(hsA*colA) # Population size
    
    return {'hsA' : hsA, 'isA' : isA, 'colA' : colA, 'rowA' : rowA, 'xA' : xA, 'N' : N}

process(SeattleA)

{'N': 177,
 'colA': array([1, 2, 3]),
 'hsA': array([26, 50, 17]),
 'isA': array([31, 32, 25,  5]),
 'rowA': array([0, 1, 2, 3]),
 'xA': 97}

In [5]:
# Code for setting (local) thresholds in a household of size n.
# 
# Note local thresholds not required for households of size 1.
# Infection rate does not depend upon households size.
#

def thresH(n):
    thres = np.repeat(0.0,n-1)
    thres[0] = np.random.exponential(size=1,scale=1/(n-1))
    if n > 2:
        for i in range(1,n-1):
            thres[i] = thres[i-1] + np.random.exponential(size=1, scale=1/(n-i))
    
    return thres

thresH(5)

array([0.31026297, 0.49516332, 1.30035441, 1.57965005])

In [174]:
def House_epi(n,k,lambda_L):
    if n == 1:
        i = 1
        if k == 0:
            sev = 1
        if k > 0:
            sev = np.random.gamma(k, 1/k, 1)
    
    if n > 1:
        t = thresH(n)
        if k == 0:
            q = np.repeat(1.0,n)
        if k > 0:
            q = np.random.gamma(size=n, shape=k, scale=1/k)
        t = np.append(t, 2*lambda_L*np.sum(q))
        
        i = 0
        test = 0
        while test == 0:
            i += 1
            if t[i-1] > (lambda_L*np.sum(q[0:i])):
                test = 1 
                sev = np.sum(q[0:i])
                
    return np.array([i,sev])

In [181]:
House_epi(1,3,0.21)

array([1.        , 0.39438582])

In [182]:
House_epi(3,5,0.21)

array([3.        , 2.45729978])

In [237]:
#
# Code for simulating a household epidemic using the Sellke construction
#

def House_SEL(hs, lambda_G, lambda_L, k):
    HH = hs.size
    ks = np.array(range(1,HH+1))

    n = np.repeat(ks,hs)

    m = n.size  # Number of households
    N = n.sum() # Population size
    NS = N

    sev = 0
    threshold = 0

    ni = np.repeat(0,m)
    ns = n.copy()

    R = np.random.exponential(size=N)

    while threshold <= lambda_G*sev:
        kk = np.random.choice(a=np.arange(m)+1, size = 1, replace = True, p = ns/(ns.sum()))[0]
        
        hou_epi = House_epi(ns[kk-1],k,lambda_L)
        ns[kk-1] = ns[kk-1] - hou_epi[0]
        sev = sev + hou_epi[1]
        
        ni[kk-1] = n[kk-1]-ns[kk-1] 
        #NS=NS-hou_epi[0]
        NS=ns.sum()
        
        if NS > 0:
            threshold = threshold + np.random.exponential(scale = NS/N, size=1)[0]
        if NS == 0:
            threshold = 2*lambda_G*sev
        
    OUT = np.zeros((HH+1,HH))
    for i in range(1, HH+2):
        for j in range(1,HH+1):
            OUT[i-1,j-1] = sum(n[ni==(i-1)]==j)
    
    return OUT

In [245]:
hs = np.array([1,2,3])
lambda_G = 0.9
lambda_L = 0.21
k=5
House_SEL(hs, lambda_G, lambda_L, k)

array([[0., 2., 3.],
       [1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [261]:
Tecumseh1 = np.zeros((8,7))
Tecumseh1[0,0:5] = np.array((66 ,87 ,  25  , 22   , 4 ))
Tecumseh1[1,0:5] = np.array((13 ,14 ,  15  ,  9   , 4 ))
Tecumseh1[2,1:6] = np.array((4 ,   4  ,  9  ,  2 ,   1 ))
Tecumseh1[3,2:7] = np.array(( 4   , 3   , 1  ,  1 ,   1))
Tecumseh1[4,3:5] = np.array(( 1,1))

In [262]:
Tecumseh1

array([[66., 87., 25., 22.,  4.,  0.,  0.],
       [13., 14., 15.,  9.,  4.,  0.,  0.],
       [ 0.,  4.,  4.,  9.,  2.,  1.,  0.],
       [ 0.,  0.,  4.,  3.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  1.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]])